In [1]:
import sys
sys.path.append('/root/jupyter/Deep_Learning/src/YOLO/')

In [2]:
from keras.models import Model
from keras.utils import Sequence
from keras.layers import Reshape, Input, Activation, Conv2D, GlobalMaxPool2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization, Flatten, Dense, Lambda, ZeroPadding2D, Dropout, DepthwiseConv2D
from keras.layers.advanced_activations import LeakyReLU, ReLU
from keras.layers.merge import concatenate
from keras.optimizers import SGD, Adam, RMSprop
from keras.callbacks import EarlyStopping, ModelCheckpoint, TensorBoard

from imgaug import augmenters as iaa
import matplotlib.pyplot as plt
import tensorflow as tf
import numpy as np
import copy
import cv2
import os

from Yolo_V2_utils import *
from Yolo_V2_extractors import *
from Yolo_V2_preprocessing import *
from Yolo_V2_detection_head import *

%matplotlib inline

Using TensorFlow backend.


In [3]:
OBJECT = 'Plate'          # Plate / Car / Number
BACKEND = 'SqueezeNet'    # DarkNet, MobileNet, SqueezeNet

ROOT_DIR = '/root/jupyter/'
DATA_DIR = ROOT_DIR + 'Deep_Learning/data/'
MODEL_DIR = ROOT_DIR + 'Deep_Learning/models/'
BACKEND_DIR = ROOT_DIR + 'Deep_Learning/backends/'

FULL_YOLO_BACKEND_PATH  = BACKEND_DIR + "yolo_v2.weights"
SQUEEZENET_BACKEND_PATH = BACKEND_DIR + "squeezenet_backend.h5"
MOBILENET_BACKEND_PATH  = BACKEND_DIR + "mobilenet_backend.h5"

TRAIN_VALID_PROP = 0.8    # Fraction des données qui seront utilisées pour l'entrainement
INPUT_SIZE = 640          # Dimension maximale de l'image
LABELS = [OBJECT]         # Liste des objets que doit YOLO considérer lors de la détection et entrainement
MAX_BOX_PER_IMAGE = 1     # GENERATED 1, MOROCCAN 26
OBJ_THRESH = 0.5          # Valeur minimale de sûreté/fidélité/confidence pour considérer la détéction positive
NMS_THRESH = 0.5          # Valeur minimale de sûreté/fidélité/confidence pour considérer la détéction positive

ANCHORS = [1.40,0.55, 2.36,0.96, 3.47,1.42, 4.47,1.93, 5.78,2.40, 6.68,3.27, 7.98,2.70, 8.58,3.70, 9.08,4.67]

# GENERATED 320 [1.40,0.55, 2.36,0.96, 3.47,1.42, 4.47,1.93, 5.78,2.40, 6.68,3.27, 7.98,2.70, 8.58,3.70, 9.08,4.67]
# GENERATED 640 [2.74,1.10, 4.39,1.83, 6.31,2.65, 8.47,3.61, 11.02,4.59, 12.80,6.18, 15.97,5.34, 16.70,7.28, 18.21,9.28]
# MOROCCAN 1024 [1.61,0.91, 5.37,2.12, 6.37,5.60, 7.61,2.80, 9.20,4.60, 9.89,3.00, 11.66,3.39, 12.68,4.25, 16.94,6.37]
# MOROCCAN RESIZED 512 [0.74,0.42, 2.28,0.93, 3.22,2.92, 3.61,1.25, 4.13,1.81, 5.12,2.64, 5.30,1.53, 6.14,1.91, 9.07,3.15]

In [4]:
model = YOLO(BACKEND, INPUT_SIZE, LABELS, MAX_BOX_PER_IMAGE, ANCHORS)

(39, 39)
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 640, 640, 3)  0                                            
__________________________________________________________________________________________________
squeezenet (Model)              (None, 39, 39, 512)  722496      input_1[0][0]                    
__________________________________________________________________________________________________
DetectionLayer (Conv2D)         (None, 39, 39, 54)   27702       squeezenet[1][0]                 
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 39, 39, 9, 6) 0           DetectionLayer[0][0]             
__________________________________________________________________________________________________
i

In [6]:
weights_file = MODEL_DIR + 'Plate_SqueezeNet_GENERATED_DATASET_Final.h5'
model.load_weights(weights_file)

In [7]:
fig=plt.figure(figsize=(20, 20))
test_imgs = os.listdir(TEST_DIR)

columns = 2
rows = 2
i = 1
passed = []

while i < rows*columns +1:
    rand_numb = np.random.randint(0,len(test_imgs)-1)
    if rand_numb in passed :
        continue
    try:
        IMAGE_PATH = TEST_DIR + test_imgs[rand_numb]
        image = cv2.imread(IMAGE_PATH)
        boxes = model.predict(image)
        image = draw_boxes(image, boxes, LABELS)
        plt_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        fig.add_subplot(rows, columns, i)
        plt.imshow(plt_image)
        passed += [rand_numb]
        i += 1
    except:
        pass

plt.show()


NameError: name 'TEST_DIR' is not defined

<Figure size 1440x1440 with 0 Axes>